In [2]:
import pandas as pd
import numpy as np
ruta='/content/drive/MyDrive/Colab Notebooks/'

data=pd.read_csv(ruta+'temps.txt',sep='\s+',header=None) # Fuente: http://astro.iam.udg.mx/Estacion/NOAAPRMO.TXT
                                                         #         http://astro.iam.udg.mx/Estacion/NOAAMO.TXT
data.columns=['Temperatura máxima','Temperatura mínima','HDD','CDD','Rain']
d=['Sábado','Domingo','Lunes','Martes','Miércoles','Jueves','Viernes']
dias=[d[i%7] for i in range(len(data))]
catego=['Soleado','Mayormente soleado','Parcialmente nublado','Mayormente nublado','Nublado','Lluvioso']
prono=[catego[-1] if data['Rain'][i]>0 else catego[int(round(1.2*(data['HDD'][i]-data['CDD'][i])+2))] for i in range(len(data))]
df=pd.DataFrame(columns=['Días','Pronóstico','Temperatura máxima','Temperatura mínima'],index=range(len(data)))
df.iloc[:,2:4]=data.iloc[:,0:2]
df['Días']=dias
df['Pronóstico']=prono

# Promedio de temperaturas para la variable `Pronóstico`

In [4]:
# Promediamos la temperatura mínima que corresponde a cada categoría de la variable 'Pronóstico'
promedio=df.groupby('Pronóstico')['Temperatura mínima'].mean()
promedio

,Temperatura mínima
Pronóstico,
Lluvioso,12.85
Mayormente nublado,10.683333
Mayormente soleado,11.435714
Nublado,9.5
Parcialmente nublado,11.261111
Soleado,11.8


In [5]:
df['Pronóstico_num']=df['Pronóstico'].map(promedio)
df.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,Pronóstico_num
0,Sábado,Parcialmente nublado,28.2,11.2,11.261111
1,Domingo,Parcialmente nublado,29.1,10.7,11.261111
2,Lunes,Mayormente nublado,26.1,10.4,10.683333
3,Martes,Mayormente soleado,29.4,11.1,11.435714
4,Miércoles,Mayormente soleado,29.8,11.1,11.435714


# Regresión lineal entre la codificación de Efectos de la variable `Pronóstico` y las temperaturas

In [6]:
CR7='Parcialmente nublado'  # Categoría de referencia 7
E=pd.get_dummies(df['Pronóstico'],prefix='',prefix_sep='').drop(columns=[CR7])
E.head() # Codificación Ficticia

,Lluvioso,Mayormente nublado,Mayormente soleado,Nublado,Soleado
0,False,False,False,False,False
1,False,False,False,False,False
2,False,True,False,False,False
3,False,False,True,False,False
4,False,False,True,False,False


In [7]:
E=E.astype('int8')
E[df['Pronóstico']==CR7]=-1
E.head()

,Lluvioso,Mayormente nublado,Mayormente soleado,Nublado,Soleado
0,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1
2,0,1,0,0,0
3,0,0,1,0,0
4,0,0,1,0,0


In [8]:
# Regresión lineal
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(E,df['Temperatura mínima'])

LinearRegression()

In [12]:
# Orden de categorías
E.columns

Index(['Lluvioso', 'Mayormente nublado', 'Mayormente soleado', 'Nublado',
       'Soleado'],
      dtype='object')

In [9]:
# Coeficientes
reg.coef_

array([ 1.59497354, -0.57169312,  0.18068783, -1.75502646,  0.54497354])

In [13]:
CR7,reg.intercept_

('Parcialmente nublado', 11.255026455026455)

In [15]:
promedio_E={categoría:media for categoría,media in zip(E.columns,reg.coef_)}
promedio_E[CR7]=reg.intercept_
promedio_E

{'Lluvioso': 1.5949735449735465,
 'Mayormente nublado': -0.5716931216931215,
 'Mayormente soleado': 0.18068783068783062,
 'Nublado': -1.7550264550264567,
 'Soleado': 0.5449735449735456,
 'Parcialmente nublado': 11.255026455026455}

In [16]:
df['Pronóstico_Efectos']=df['Pronóstico'].map(promedio_E)
df.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,Pronóstico_num,Pronóstico_Efectos
0,Sábado,Parcialmente nublado,28.2,11.2,11.261111,11.255026
1,Domingo,Parcialmente nublado,29.1,10.7,11.261111,11.255026
2,Lunes,Mayormente nublado,26.1,10.4,10.683333,-0.571693
3,Martes,Mayormente soleado,29.4,11.1,11.435714,0.180688
4,Miércoles,Mayormente soleado,29.8,11.1,11.435714,0.180688
